# Třetí cvičení - jazykové modely, perplexita

## 1) Příprava dat

- Soubory TEXTCZ1 a TEXTEN1 přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase

In [1]:
import re, string
from collections import Counter
from math import log2
import math

## Unigramový model
- Vypočítejte pravděpodobnost každého ze slov
- Vypočítejte entropii H(X) a perplexitu G(X) modelu

$H(X) = - \sum_{x}p(x)log{_2}p(x)$

$G(X) = 2^{H(X)}$

In [2]:
# Nejprve vytvořte unigramový model pro větu: "<s> Dnes je hezký den"
sent = "<s> Dnes je hezký den"

words = sent.split(' ')
words_counts = Counter(words)
words_count = len(words_counts)

entropy = -sum([(words_counts[x]/words_count) * log2(words_counts[x]/words_count) for x in words_counts])
print(f'entropy = {entropy}')
perplexity = 2**entropy
print(f'perplexity = {perplexity}')

# Očekávaný výsledek: 
# entropy = 2.321928094887362
# perplexity = 4.999999999999999

entropy = 2.321928094887362
perplexity = 4.999999999999999


In [3]:
# Nyní pro CZ a EN korpusy

In [4]:
# Načtení souborů CZ, EN
text_cz = []
with open('TEXTCZ1.txt', 'r', encoding='cp1250') as file_cz:
    while line := file_cz.readline():
        text_cz.append(line)

text_en = []
with open('TEXTEN1.txt', 'r', encoding='UTF-8') as file_en:
    while line := file_en.readline():
        text_en.append(line)
        

In [13]:
# Rozdělení textů na věty, odstranění interpunkce a přidání znaku začátku věty
def text_to_lines(text):
    text_lined = []
    line = ["<s>"]
    for word in text:
        if word.strip() in ".?!":
            text_lined.append(line)
            line = ["<s>"]
            continue
        if word.strip() in string.punctuation:
            continue
        line.append(word.strip().translate(str.maketrans('', '', string.punctuation)).lower())
    return text_lined

text_cz_lined = text_to_lines(text_cz)
text_en_lined = text_to_lines(text_en)

In [17]:
def word_freqs(text):
    words_counts = Counter()
    for line in text:
        words_counts.update(Counter(line))
    words_count = sum(words_counts.values())
    return words_counts, words_count

def unigram_entropy(words_freqs, words_count):
    entropy = 0
    for x in words_freqs:
        entropy += (words_freqs[x]/words_count) * log2(words_freqs[x]/words_count)
    return -entropy

In [19]:
# Anglický unigramový model
words_counts_en, words_count_en = word_freqs(text_en_lined)

en_entropy = unigram_entropy(words_counts_en, words_count_en)
en_perplexity = 2**en_entropy

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 9.153156076444484
# en_perplexity = 569.3437191681389

Entropy: 9.154048271867357 Perplexity: 569.6959231662712


In [20]:
# Český unigramový model
words_counts_cz, words_count_cz = word_freqs(text_cz_lined)

cz_entropy = unigram_entropy(words_counts_cz, words_count_cz)
cz_perplexity = 2**cz_entropy

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 11.866640196670001
# en_perplexity = 3734.346796191333


Entropy: 11.901302885729898 Perplexity: 3825.156040129388


## Bigramový model
- obdobně vytvořte bigramový model a všechny dvojice slov hledejte vždy pouze ve větě (nikoliv mezi větami)
- vypočítejte entropii a perplexitu

$H(B|A) = - \sum_{a \in A, b \in B}P(a,b)log{_2}P(b|a)$ 

$G(X) = 2^{H(B|A)}$

In [144]:
# Nejprve vytvořte bigramový model pro větu: "<s> Dnes je hezký den </s>"
sent = ["<s> dnes respektive dnes možná </s>"]
words_count = 0
pairs_counts = Counter()
words_counts = Counter()
for line in sent:
    words = line.split(' ')
    words_counts.update(Counter(words))
    pairs = [f'{words[i]} {words[i+1]}' for i in range(len(words)-1)]
    pairs_counts.update(Counter(pairs))
    words_count += len(words)-1 # -1 aby se nepocital </s>
words_count = len(words_counts)
pairs_count = sum(pairs_counts.values())

entropy = -sum([(pairs_counts[x]/pairs_count) * log2((pairs_counts[x]/pairs_count)/(words_counts[x.split(' ')[1]]/words_count)) for x in pairs_counts])
perplexity = 2**entropy

print(f"Entropy: {entropy} Perplexity: {perplexity}")
# Očekávaný výsledek: 
# entropy = 0.4
# perplexity = 1.3195079107728942

Entropy: 0.4 Perplexity: 1.3195079107728942


In [140]:
# Nyní pro jednotlivé korpusy, nejprve pro CZ a pak pro EN

# Přidání koncového znaku </s> do vět
for line in text_cz_lined:
    line.append('</s>')
for line in text_en_lined:
    line.append('</s>')

In [150]:
# Rozdělení na slova, tvorba slovníku a výpočet frekvencí
def bigrams(lines):
    pairs_counts = Counter()
    words_counts = Counter()
    for words in lines:
        words_counts.update(Counter(words))
        pairs = [f'{words[i]} {words[i+1]}' for i in range(len(words)-1)]
        pairs_counts.update(Counter(pairs))
    return pairs_counts, words_counts

In [156]:
pairs_counts, words_counts = bigrams(text_en_lined)
words_count = sum(words_counts.values())
pairs_count = sum(pairs_counts.values())

en_entropy = -sum([(pairs_counts[x]/pairs_count) * log2((pairs_counts[x]/pairs_count)/(words_counts[x.split(' ')[1]]/words_count)) for x in pairs_counts])
en_perplexity = 2**en_entropy

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")

# Očekávaný výsledek: 
# en_entropy = 5.380173547272233
# en_perplexity = 41.64794906297528

Entropy: 5.24714317366672 Perplexity: 37.97934642441261


In [158]:
pairs_counts, words_counts = bigrams(text_cz_lined)
words_count = sum(words_counts.values())
pairs_count = sum(pairs_counts.values())

cz_entropy = -sum([(pairs_counts[x]/pairs_count) * log2((pairs_counts[x]/pairs_count)/(words_counts[x.split(' ')[1]]/words_count)) for x in pairs_counts])
cz_perplexity = 2**cz_entropy

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")

# Očekávaný výsledek: 
# cz_entropy = 4.624183207248371
# cz_perplexity = 24.6614070105357

Entropy: 4.331367509109961 Perplexity: 20.131287098126155


In [ ]:
#SRILM příkazy
#ngram-count -text TEXTEN_unigram.txt -order 2 -write czbigram.count -unk
#ngram-count -gt1max 0 -gt2max 0 -read czbigram.count -order 2 -lm czbigram.lm
#ngram -ppl TEXTEN_unigram.txt -order 2 -lm czbigram.lm >> czbigram.ppl

